# Model Explainability

#### Explaining BCI AVM's XGBoost Tree Model with Path-Dependent Feature Perturbation using Tree SHAP

#### About SHAP Values

SHAP (SHapley Additive exPlanations) is a game theoretic approach to explain the output of any machine learning model. It connects optimal credit allocation with local explanations using the classic Shapley values from game theory and their related extensions (see papers for details and citations).


<p align="center">
<img width=25% src="https://blockchainclimate.org/wp-content/uploads/2020/11/cropped-BCI_Logo_LR-400x333.png" alt="bciAVM" height="300"/>
</p>

[![PyPI](https://badge.fury.io/py/bciavm.svg?maxAge=2592000)](https://badge.fury.io/py/bciavm)
[![PyPI Stats](https://img.shields.io/badge/bciavm-avm-blue)](https://pypistats.org/packages/bciavm)


This notebook contains code to take a `mlflow` registered model and distribute its work with a `Dask` cluster. 
<table>
    <tr>
        <td>
            <img width=25% src="https://saturn-public-assets.s3.us-east-2.amazonaws.com/example-resources/dask.png" width="300">
        </td>
    </tr>
</table>

The Blockchain & Climate Institute (BCI) is a progressive think tank providing leading expertise in the deployment of emerging technologies for climate and sustainability actions. 

As an international network of scientific and technological experts, BCI is at the forefront of innovative efforts, enabling technology transfers, to create a sustainable and clean global future.

# Automated Valuation Model (AVM) 

### About
AVM is a term for a service that uses mathematical modeling combined with databases of existing properties and transactions to calculate real estate values. 
The majority of automated valuation models (AVMs) compare the values of similar properties at the same point in time. 
Many appraisers, and even Wall Street institutions, use this type of model to value residential properties. (see [What is an AVM](https://www.investopedia.com/terms/a/automated-valuation-model.asp) Investopedia.com)

For more detailed info about the AVM, please read the **About** paper found here `resources/2021-BCI-AVM-About.pdf`.

### Valuation Process
<img src="resources/valuation_process.png" height="360" >

**Key Functionality**

* **Supervised algorithms** 
* **Tree-based & deep learning algorithms** 
* **Feature engineering derived from small clusters of similar properties** 
* **Ensemble (value blending) approaches** 

### Set the required AWS Environment Variables
```shell
export ACCESS_KEY=YOURACCESS_KEY
export SECRET_KEY=YOURSECRET_KEY
export BUCKET_NAME=bci-transition-risk-data
export TABLE_DIRECTORY=/dbfs/FileStore/tables/
```

### Next Steps
Read more about bciAVM on our [documentation page](https://blockchainclimate.org/thought-leadership/#blog):

### How does it relate to BCI Risk Modeling?
<img src="resources/bci_flowchart_2.png" height="280" >


### Technical & financial support for development provided by:
<a href="https://www.gcode.ai">
    <img width=15% src="https://staticfiles-img.s3.amazonaws.com/avm/gcode_logo.png" alt="GCODE.ai"  height="25"/>
</a>


### Install [from PyPI](https://pypi.org/project/bciavm/)
```shell
pip install bciavm
```

This notebook covers the following steps:
- Import data from your local machine into the Databricks File System (DBFS)
- Download data from s3
- Train a machine learning models (or more technically, multiple models in a stacked pipeline) on the dataset
- Register the model in MLflow

In [2]:
import bciavm, shap
import numpy as np
import pandas as pd
from functools import partial

x_path, y_path = '/dbfs/FileStore/tables/avm/X_test.csv', '/dbfs/FileStore/tables/avm/y_test.csv'
pipe_path = '/dbfs/FileStore/artifacts/avm_pipeline_'+str(bciavm.__version__)+'.pkl'
target = 'Price_p'
data_index = 'unit_indx'

In [3]:
pipeline = bciavm.pipelines.RegressionPipeline.load(pipe_path)
model = pipeline.get_component('XGB Regressor')._component_obj
model

Out[3]: XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
 colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
 importance_type='gain', interaction_constraints='',
 learning_rate=0.1040805594257481, max_delta_step=0, max_depth=11,
 min_child_weight=6, missing=nan, monotone_constraints='()',
 n_estimators=260, n_jobs=4, num_parallel_tree=1, random_state=0,
 reg_alpha=0.04216890859017286, reg_lambda=0.2541818897259287,
 scale_pos_weight=1, subsample=1, tree_method='exact',
 validate_parameters=1, verbosity=None)

In [4]:
#the avm model features
features = pipeline.get_component('XGB Regressor').input_feature_names
print('# of features = ', len(features))
features

# of features = 28
Out[4]: ['TOTAL_FLOOR_AREA_e',
 'NUMBER_HEATED_ROOMS_e',
 'FLOOR_LEVEL_e',
 'Latitude_m',
 'Longitude_m',
 'density_count',
 'Price_p__mean',
 'Price_p__std',
 'Price_p__min',
 'Price_p__median',
 'Price_p__max',
 'TOTAL_FLOOR_AREA_e_minus_mean',
 'TOTAL_FLOOR_AREA_e_minus_std',
 'TOTAL_FLOOR_AREA_e_minus_min',
 'TOTAL_FLOOR_AREA_e_minus_median',
 'TOTAL_FLOOR_AREA_e_minus_max',
 'NUMBER_HEATED_ROOMS_e_minus_mean',
 'NUMBER_HEATED_ROOMS_e_minus_std',
 'NUMBER_HEATED_ROOMS_e_minus_min',
 'NUMBER_HEATED_ROOMS_e_minus_median',
 'NUMBER_HEATED_ROOMS_e_minus_max',
 'FLOOR_LEVEL_e_minus_mean',
 'FLOOR_LEVEL_e_minus_std',
 'FLOOR_LEVEL_e_minus_min',
 'FLOOR_LEVEL_e_minus_median',
 'FLOOR_LEVEL_e_minus_max',
 'agg_cat_B_House_5.0',
 'agg_cat_S_House_5.0']

In [5]:
X = pd.read_csv(x_path)
y = pd.read_csv(y_path)

#create the explainer
explainer = shap.TreeExplainer(model, feature_perturbation="tree_path_dependent")

# define charts
def _dependence_plot(features, shap_values, dataset, feature_names, fig_height=15, fig_width=10,  display_features=None, **kwargs):
    """ 
    Plots dependence plots of specified features in a grid.
    
    features: List[str], List[Tuple[str, str]]
        Names of features to be plotted. If List[str], then shap 
        values are plotted as a function of feature value, coloured 
        by the value of the feature determined to have the strongest
        interaction (empirically). If List[Tuple[str, str]], shap
        interaction values are plotted.
    display_features: np.ndarray, N x F
        Same as dataset, but contains human readable values
        for categorical levels as opposed to numerical values
    """
    
    def _set_fonts(fig, ax, fonts=None, set_cbar=False):
        """
        Sets fonts for axis labels and colobar.
        """

        ax.xaxis.label.set_size(xlabelfontsize)
        ax.yaxis.label.set_size(ylabelfontsize)
        ax.tick_params(axis='x', labelsize=xtickfontsize)
        ax.tick_params(axis='y', labelsize=ytickfontsize)
        if set_cbar:
            fig.axes[-1].tick_params(labelsize=cbartickfontsize)
            fig.axes[-1].tick_params(labelrotation=cbartickrotation)
            fig.axes[-1].yaxis.label.set_size(cbarlabelfontsize)

    # parse plotting args
    figsize = kwargs.get("figsize", (fig_height, fig_width))
    nrows = kwargs.get('nrows', len(features))
    ncols = kwargs.get('ncols', 1)
    xlabelfontsize = kwargs.get('xlabelfontsize', 14)
    xtickfontsize = kwargs.get('xtickfontsize', 11)
    ylabelfontsize = kwargs.get('ylabelfontsize', 14)
    ytickfontsize = kwargs.get('ytickfontsize', 11)
    cbartickfontsize = kwargs.get('cbartickfontsize', 14)
    cbartickrotation = kwargs.get('cbartickrotation', 45)
    cbarlabelfontsize = kwargs.get('cbarlabelfontsize', 14)
    rotation_orig = kwargs.get('xticklabelrotation', 25)
    alpha = kwargs.get("alpha", 1)
    x_jitter_orig = kwargs.get("x_jitter", 0.8)
    grouped_features = list(zip_longest(*[iter(features)] * ncols))
    
    fig, axes = plt.subplots(nrows, ncols,  figsize=figsize)
    if nrows == len(features):
        axes = list(zip_longest(*[iter(axes)] * 1))


    for i, (row, group) in enumerate(zip(axes, grouped_features), start=1):
        # plot each feature or interaction in a subplot
        for ax, feature in zip(row, group):
            # set x-axis ticks and labels and x-jitter for categorical variables
            if not feature:
                continue
            if isinstance(feature, list) or isinstance(feature, tuple):
                feature_index = feature_names.index(feature[0])
            else:
                feature_index = feature_names.index(feature)

            x_jitter = 0
            
            shap.dependence_plot(feature, 
                                 shap_values,
                                 dataset,
                                 feature_names=feature_names,
                                 display_features=display_features,
                                 interaction_index='auto',
                                 ax=ax,
                                 show=False,
                                 x_jitter=x_jitter,
                                 alpha=alpha
                                )
            if i!= nrows:
                ax.tick_params('x', labelrotation=rotation_orig)
            _set_fonts(fig, ax, set_cbar=True)
    

def plot_decomposition(feature_pair, shap_interaction_vals, features, feat_names, display_features=None, **kwargs):
    """
    Given a list containing two feature names (`feature_pair`), an n_instances x n_features x n_features tensor 
    of shap interaction values (`shap_interaction_vals`), an n_instances x n_features (`features`) tensor of 
    feature values and a list of feature names (which assigns a name to each column of `features`), this function 
    plots:
        - left: shap values for feature_pair[0] coloured by the value of feature_pair[1]
        - middle: shap values for feature_pair[0] after subtracting the interaction with feature_pair[1]
        - right: the interaction values between feature_pair[0] and feature_pair[1], which are subtracted 
        from the left plot to get the middle plot
        
    NB: `display_features` is the same shape as `features` but should contain the raw categories for categorical 
    variables so that the colorbar can be discretised and the category names displayed alongside the colorbar.
    """
    
    def _set_fonts(fig, ax, fonts=None, set_cbar=False):
        """
        Sets fonts for axis labels and colobar.
        """

        ax.xaxis.label.set_size(xlabelfontsize)
        ax.yaxis.label.set_size(ylabelfontsize)
        ax.tick_params(axis='x', labelsize=xtickfontsize)
        ax.tick_params(axis='y', labelsize=ytickfontsize)
        if set_cbar:
            fig.axes[-1].tick_params(labelsize=cbartickfontsize)
            fig.axes[-1].yaxis.label.set_size(cbarlabelfontsize)

    # parse plotting args
    xlabelfontsize = kwargs.get('xlabelfontsize', 21)
    ylabelfontsize = kwargs.get('ylabelfontsize', 16)
    cbartickfontsize = kwargs.get('cbartickfontsize', 16)
    cbarlabelfontsize = kwargs.get('cbarlabelfontsize', 21)
    xtickfontsize = kwargs.get('xtickfontsize', 20)
    ytickfontsize = kwargs.get('ytickfontsize', 16)
    alpha = kwargs.get('alpha', 0.7)
    figsize = kwargs.get('figsize', (44, 16))
    ncols = kwargs.get('ncols', 3)
    nrows = kwargs.get('nrows', 1)
    
    # compute shap values and shap values without interaction
    feat1_idx = feat_names.index(feature_pair[0])
    feat2_idx = feat_names.index(feature_pair[1])
    
    # shap values
    shap_vals = shap_interaction_vals.sum(axis=2)
    
    # shap values for feat1, all samples
    shap_val_ind1 = shap_interaction_vals[..., feat1_idx].sum(axis=1)
    
    # shap values for (feat1, feat2) interaction 
    shap_int_ind1_ind2 = shap_interaction_vals[:, feat2_idx, feat1_idx]
    
    # subtract effect of feat2
    shap_val_minus_ind2 = shap_val_ind1 - shap_int_ind1_ind2
    shap_val_minus_ind2 = shap_val_minus_ind2[:, None]

    # create plot
    fig, (ax1, ax2, ax3) = plt.subplots(nrows, ncols, figsize=figsize)

    # plot the shap values including the interaction
    shap.dependence_plot(feature_pair[0],
                         shap_vals,
                         features,
                         display_features = display_features,
                         feature_names=feat_names,
                         interaction_index=feature_pair[1],
                         alpha=alpha,
                         ax=ax1,
                         show=False)
    _set_fonts(fig, ax1, set_cbar=True)

    # plot the shap values excluding the interaction
    shap.dependence_plot(0,
                         shap_val_minus_ind2,
                         features[:, feat1_idx][:, None],
                         feature_names=[feature_pair[0]],
                         interaction_index=None,
                         alpha=alpha,
                         ax=ax2,
                         show=False,
                         )
    ax2.set_ylabel(f' Shap value for  {feature_pair[0]} \n wo {feature_pair[1]} interaction')
    _set_fonts(fig, ax2)
    
    # plot the interaction value
    shap.dependence_plot(feature_pair,
                         shap_interaction_vals,
                         features,
                         feature_names=feat_names,
                         display_features=display_features,
                         interaction_index='auto',
                         alpha=alpha,
                         ax=ax3,
                         show=False,
                        )
    _set_fonts(fig, ax3, set_cbar=True)
    



def plot_decomposition(feature_pair, shap_interaction_vals, features, feat_names, display_features=None, **kwargs):
    """
    Given a list containing two feature names (`feature_pair`), an n_instances x n_features x n_features tensor 
    of shap interaction values (`shap_interaction_vals`), an n_instances x n_features (`features`) tensor of 
    feature values and a list of feature names (which assigns a name to each column of `features`), this function 
    plots:
        - left: shap values for feature_pair[0] coloured by the value of feature_pair[1]
        - middle: shap values for feature_pair[0] after subtracting the interaction with feature_pair[1]
        - right: the interaction values between feature_pair[0] and feature_pair[1], which are subtracted 
        from the left plot to get the middle plot
        
    NB: `display_features` is the same shape as `features` but should contain the raw categories for categorical 
    variables so that the colorbar can be discretised and the category names displayed alongside the colorbar.
    """
    
    def _set_fonts(fig, ax, fonts=None, set_cbar=False):
        """
        Sets fonts for axis labels and colobar.
        """

        ax.xaxis.label.set_size(xlabelfontsize)
        ax.yaxis.label.set_size(ylabelfontsize)
        ax.tick_params(axis='x', labelsize=xtickfontsize)
        ax.tick_params(axis='y', labelsize=ytickfontsize)
        if set_cbar:
            fig.axes[-1].tick_params(labelsize=cbartickfontsize)
            fig.axes[-1].yaxis.label.set_size(cbarlabelfontsize)

    # parse plotting args
    xlabelfontsize = kwargs.get('xlabelfontsize', 21)
    ylabelfontsize = kwargs.get('ylabelfontsize', 16)
    cbartickfontsize = kwargs.get('cbartickfontsize', 16)
    cbarlabelfontsize = kwargs.get('cbarlabelfontsize', 21)
    xtickfontsize = kwargs.get('xtickfontsize', 20)
    ytickfontsize = kwargs.get('ytickfontsize', 16)
    alpha = kwargs.get('alpha', 0.7)
    figsize = kwargs.get('figsize', (44, 16))
    ncols = kwargs.get('ncols', 3)
    nrows = kwargs.get('nrows', 1)
    # compute shap values and shap values without interaction
    feat1_idx = feat_names.index(feature_pair[0])
    feat2_idx = feat_names.index(feature_pair[1])
    # shap values
    shap_vals = shap_interaction_vals.sum(axis=2)
    # shap values for feat1, all samples
    shap_val_ind1 = shap_interaction_vals[..., feat1_idx].sum(axis=1)
    # shap values for (feat1, feat2) interaction 
    shap_int_ind1_ind2 = shap_interaction_vals[:, feat2_idx, feat1_idx]
    # subtract effect of feat2
    shap_val_minus_ind2 = shap_val_ind1 - shap_int_ind1_ind2
    shap_val_minus_ind2 = shap_val_minus_ind2[:, None]

    # create plot

    fig, (ax1, ax2, ax3) = plt.subplots(nrows, ncols, figsize=figsize)

    # plot the shap values including the interaction
    shap.dependence_plot(feature_pair[0],
                         shap_vals,
                         features,
                         display_features = display_features,
                         feature_names=feat_names,
                         interaction_index=feature_pair[1],
                         alpha=alpha,
                         ax=ax1,
                         show=False)
    _set_fonts(fig, ax1, set_cbar=True)

    # plot the shap values excluding the interaction
    shap.dependence_plot(0,
                         shap_val_minus_ind2,
                         features[:, feat1_idx][:, None],
                         feature_names=[feature_pair[0]],
                         interaction_index=None,
                         alpha=alpha,
                         ax=ax2,
                         show=False,
                         )
    ax2.set_ylabel(f' Shap value for  {feature_pair[0]} \n wo {feature_pair[1]} interaction')
    _set_fonts(fig, ax2)
    
    # plot the interaction value
    shap.dependence_plot(feature_pair,
                         shap_interaction_vals,
                         features,
                         feature_names=feat_names,
                         display_features=display_features,
                         interaction_index='auto',
                         alpha=alpha,
                         ax=ax3,
                         show=False)
    
    _set_fonts(fig, ax3, set_cbar=True)
    


In [6]:
preprocessor = pipeline.get_component("Preprocess Transformer")
imputer = pipeline.get_component('Imputer')
ohe = pipeline.get_component('One Hot Encoder')

In [7]:
X_processed = X.copy()
X_t = preprocessor.transform(X_processed)
X_t = imputer.transform(X_t)
X_t = ohe.transform(X_t)
X_t = X_t[features]
X_t

Out[7]:

,TOTAL_FLOOR_AREA_e,NUMBER_HEATED_ROOMS_e,FLOOR_LEVEL_e,Latitude_m,Longitude_m,density_count,Price_p__mean,Price_p__std,Price_p__min,Price_p__median,Price_p__max,TOTAL_FLOOR_AREA_e_minus_mean,TOTAL_FLOOR_AREA_e_minus_std,TOTAL_FLOOR_AREA_e_minus_min,TOTAL_FLOOR_AREA_e_minus_median,TOTAL_FLOOR_AREA_e_minus_max,NUMBER_HEATED_ROOMS_e_minus_mean,NUMBER_HEATED_ROOMS_e_minus_std,NUMBER_HEATED_ROOMS_e_minus_min,NUMBER_HEATED_ROOMS_e_minus_median,NUMBER_HEATED_ROOMS_e_minus_max,FLOOR_LEVEL_e_minus_mean,FLOOR_LEVEL_e_minus_std,FLOOR_LEVEL_e_minus_min,FLOOR_LEVEL_e_minus_median,FLOOR_LEVEL_e_minus_max,agg_cat_B_House_5.0,agg_cat_S_House_5.0
0,75.0,4.0,1.0,51.455688,-0.161738,6.0,212000.0,40305.086528,155750.0,205000.0,280000.0,-2.043333,66.740622,15.00,0.000,-25.0,0.00,3.500000,1.0,0.0,-1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,103.0,6.0,0.0,52.912711,-1.144708,2.0,320000.0,21213.203436,305000.0,320000.0,335000.0,0.500000,92.393398,8.00,0.500,-7.0,0.50,5.292893,1.0,0.5,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,77.0,5.0,0.0,51.425884,-0.423289,8.0,429000.0,126471.905406,325000.0,387500.0,705000.0,-12.506250,53.299150,11.80,-5.000,-67.0,0.50,3.585786,3.0,0.5,-2.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,90.0,4.0,0.0,53.838379,-1.547717,6.0,212000.0,40305.086528,155750.0,205000.0,280000.0,-2.043333,66.740622,15.00,0.000,-25.0,0.00,3.500000,1.0,0.0,-1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,75.0,3.0,1.0,52.584828,-0.225782,5.0,69200.0,24724.987361,42500.0,65000.0,110000.0,28.200000,58.531849,38.00,38.000,0.0,0.80,2.552786,1.0,1.0,0.0,-0.4,-0.140175,1.0,0.0,-2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109809,100.0,5.0,0.0,54.888152,-2.941256,6.0,212000.0,40305.086528,155750.0,205000.0,280000.0,-2.043333,66.740622,15.00,0.000,-25.0,0.00,3.500000,1.0,0.0,-1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
109810,229.0,9.0,0.0,54.134593,-0.808554,6.0,212000.0,40305.086528,155750.0,205000.0,280000.0,-2.043333,66.740622,15.00,0.000,-25.0,0.00,3.500000,1.0,0.0,-1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
109811,66.0,3.0,0.0,50.206130,-5.385605,6.0,212000.0,40305.086528,155750.0,205000.0,280000.0,-2.043333,66.740622,15.00,0.000,-25.0,0.00,3.500000,1.0,0.0,-1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
109812,191.0,5.0,0.0,53.381240,-2.318664,2.0,347500.0,102530.483272,275000.0,347500.0,420000.0,53.225000,115.728483,106.45,53.225,0.0,0.00,5.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [8]:
#Use a small sample bc SHAP is slow
Xs = X_t.sample(100)

# #calculate the shap values
shap_values = explainer.shap_values(Xs.values, check_additivity=False)

# #calculate the interaction values
shap_interaction_values = explainer.shap_interaction_values(Xs.values)


### Figure 1: Features that lower the AVM valuation
- A lower `TOTAL_FLOOR_AREA_e` feature is generally indicative of a lower AVM valuation
- A lower `Price_p__median` feature is generally indicative of a lower AVM valuation
- A lower `Price_p__mean` is generally indicative of a lower AVM value
- A higher `Latitude_m` is generally indicative of a lower AVM value

In [10]:
shap.summary_plot(shap_values, Xs, features)

### Figure 2: Features that increase the AVM valuation
- The inverse of the above is also true.

In [12]:
shap.summary_plot(shap_values, Xs, features)

### Figure 3: Interesting Partial Dependence Plots

While the above SHAP summary plots give a general overview of each feature, the below SHAP dependence plots show how the model output varies by feauture value. 

In the Summary Plots, note that every dot is a `unit_indx` valuation, and the vertical dispersion at a single feature value results from interaction effects in the model. The feature used for coloring is automatically chosen to highlight what might be driving these interactions. Later we will see how to check that the interaction is really in the model with SHAP interaction values. Note that the row of a SHAP summary plot results from projecting the points of a SHAP dependence plot onto the y-axis, then recoloring by the feature itself.

Below we give the SHAP dependence plot for each of the chosen features, revealing interesting but expected trends. Keep in mind the calibration of some of these values can be different than a real world effect, so it is wise to be careful drawing concrete conclusions.

- **(top left)** 

- **(top right)** 

- **(bottom left)** 

- **(bottom right)**

In [14]:
from itertools import product, zip_longest
import matplotlib.pyplot as plt

plot_dependence = partial(
    _dependence_plot, 
    feature_names=features,
    category_map={},
)

plot_dependence(
    [ 
     'TOTAL_FLOOR_AREA_e',
     'NUMBER_HEATED_ROOMS_e', 
     'FLOOR_LEVEL_e', 
     'Price_p__median'
    ], 
    shap_values, 
    Xs, 
    display_features = Xs,  
    rotation=33,
    figsize=(40, 20), 
    alpha=1, 
    x_jitter=0.5,
    nrows=2,
    ncols=2,
    xlabelfontsize=24,
    xtickfontsize=20,
    xticklabelrotation=0,
    ylabelfontsize=24,
    ytickfontsize=21,
    cbarlabelfontsize=22,
    cbartickfontsize=20,
    cbartickrotation=0
)

#### Figure 4: Percent Error By Price Range

We can calculate and graph the % Error by Price Range as follows:

In [16]:

def _plot_accuracy_by_price_range(y_test, y_stacked, units):
    model_name='avm'

    df_acc = pd.DataFrame({'y_test': y_test, 'preds': y_stacked, 'unit_indx': units}, columns=['y_test', 'preds', 'unit_indx'])
    df_acc['error'] = abs((df_acc['preds'] - df_acc['y_test']) / df_acc['y_test'])
    df_acc['percent_error'] = (df_acc['preds'] - df_acc['y_test']) / df_acc['y_test']

    ctt = []
    try: 
        if len(df_acc[df_acc['y_test']<=50000]) > 5: ctt.append('df_0')
    except: pass
    try: 
        if len(df_acc[(df_acc['y_test']>=50000) & (df_acc['y_test']<=100000)]) > 5: ctt.append('df_5')
    except: pass
    try: 
        if len(df_acc[(df_acc['y_test']>=100000) & (df_acc['y_test']<=150000)]) > 5: ctt.append('df_10')
    except: pass
    try: 
        if len(df_acc[(df_acc['y_test']>=150000) & (df_acc['y_test']<=200000)]) > 5: ctt.append('df_15')
    except: pass
    try: 
        if len(df_acc[(df_acc['y_test']>=200000) & (df_acc['y_test']<=250000)]) > 5: ctt.append('df_20')
    except: pass
    try: 
        if len(df_acc[(df_acc['y_test']>=250000) & (df_acc['y_test']<=300000)]) > 5: ctt.append('df_25')
    except: pass
    try: 
        if len(df_acc[(df_acc['y_test']>=300000) & (df_acc['y_test']<=350000)]) > 5: ctt.append('df_30')
    except: pass
    try: 
        if len(df_acc[(df_acc['y_test']>=350000) & (df_acc['y_test']<=400000)]) > 5: ctt.append('df_35')
    except: pass
    try: 
        if len(df_acc[df_acc['y_test']>=400000]) > 5: ctt.append('df_40')
    except: pass

    def troubleshoot(ctt, df_acc):

        acc_2 = len(ctt)*[0]
        acc_5 = len(ctt)*[0]
        acc_10 = len(ctt)*[0]
        acc_20 = len(ctt)*[0]

        x = 0
        if 'df_0' in ctt:
            try:
                df_0 = df_acc[df_acc['y_test']<=50000]
                acc_2[x] = len(df_0[df_0['error']<0.02])/len(df_0)
                acc_5[x] = len(df_0[df_0['error']<0.05])/len(df_0)
                acc_10[x] = len(df_0[df_0['error']<0.1])/len(df_0)
                acc_20[x] = len(df_0[df_0['error']<0.2])/len(df_0)
                x += 1
            except: 
                ctt.pop('df_0')
                troubleshoot(ctt, df_acc)

        if 'df_5' in ctt:
            try:
                df_5 = df_acc[(df_acc['y_test']>=50000) & (df_acc['y_test']<=100000)]
                acc_2[x] = len(df_5[df_5['error']<0.02])/len(df_5)
                acc_5[x] = len(df_5[df_5['error']<0.05])/len(df_5)
                acc_10[x] = len(df_5[df_5['error']<0.10])/len(df_5)
                acc_20[x] = len(df_5[df_5['error']<0.2])/len(df_5)
                x += 1
            except: 
                ctt.pop('df_5')
                troubleshoot(ctt, df_acc)

        if 'df_10' in ctt:
            try:
                df_10 = df_acc[(df_acc['y_test']>=100000) & (df_acc['y_test']<=150000)]
                acc_2[x] = len(df_10[df_10['error']<0.02])/len(df_10)
                acc_5[x] = len(df_10[df_10['error']<0.05])/len(df_10)
                acc_10[x] = len(df_10[df_10['error']<0.10])/len(df_10)
                acc_20[x] = len(df_10[df_10['error']<0.2])/len(df_10)
                x += 1
            except: 
                ctt.pop('df_10')
                troubleshoot(ctt, df_acc)

        if 'df_10' in ctt:
            try:
                df_15 = df_acc[(df_acc['y_test']>=150000) & (df_acc['y_test']<=200000)]
                acc_2[x] = len(df_15[df_15['error']<0.02])/len(df_15)
                acc_5[x] = len(df_15[df_15['error']<0.05])/len(df_15)
                acc_10[x] = len(df_15[df_15['error']<0.1])/len(df_15)
                acc_20[x] = len(df_15[df_15['error']<0.2])/len(df_15)
                x += 1
            except: 
                ctt.pop('df_10')
                troubleshoot(ctt, df_acc)

        if 'df_20' in ctt:
            try:
                df_20 = df_acc[(df_acc['y_test']>=200000) & (df_acc['y_test']<=250000)]
                acc_2[x] = len(df_20[df_20['error']<0.02])/len(df_20)
                acc_5[x] = len(df_20[df_20['error']<0.05])/len(df_20)
                acc_10[x] = len(df_20[df_20['error']<0.1])/len(df_20)
                acc_20[x] = len(df_20[df_20['error']<0.2])/len(df_20)
                x += 1
            except: 
                ctt.pop('df_20')
                troubleshoot(ctt, df_acc)

        if 'df_25' in ctt:
            try:
                df_25 = df_acc[(df_acc['y_test']>=250000) & (df_acc['y_test']<=300000)]
                acc_2[x] = len(df_25[df_25['error']<0.02])/len(df_25)
                acc_5[x] = len(df_25[df_25['error']<0.05])/len(df_25)
                acc_10[x] = len(df_25[df_25['error']<0.1])/len(df_25)
                acc_20[x] = len(df_25[df_25['error']<0.2])/len(df_25)
                x += 1
            except: 
                ctt.pop('df_25')
                troubleshoot(ctt, df_acc)

        if 'df_30' in ctt:
            try:
                df_30 = df_acc[(df_acc['y_test']>=300000) & (df_acc['y_test']<=350000)]
                acc_2[x] = len(df_30[df_30['error']<0.02])/len(df_30)
                acc_5[x] = len(df_30[df_30['error']<0.05])/len(df_30)
                acc_10[x] = len(df_30[df_30['error']<0.1])/len(df_30)
                acc_20[x] = len(df_30[df_30['error']<0.2])/len(df_30)
                x += 1
            except: 
                ctt.pop('df_30')
                troubleshoot(ctt, df_acc)

        if 'df_35' in ctt:
            try:
                df_35 = df_acc[(df_acc['y_test']>=350000) & (df_acc['y_test']<=400000)]
                acc_2[x] = len(df_35[df_35['error']<0.02])/len(df_35)
                acc_5[x] = len(df_35[df_35['error']<0.05])/len(df_35)
                acc_10[x] = len(df_35[df_35['error']<0.1])/len(df_35)
                acc_20[x] = len(df_35[df_35['error']<0.2])/len(df_35)
                x += 1
            except: 
                ctt.pop('df_35')
                troubleshoot(ctt, df_acc)

        if 'df_40' in ctt:
            try:
                df_40 = df_acc[df_acc['y_test']>=400000]
                acc_2[x] = len(df_40[df_40['error']<0.02])/len(df_40)
                acc_5[x] = len(df_40[df_40['error']<0.05])/len(df_40)
                acc_10[x] = len(df_40[df_40['error']<0.1])/len(df_40)
                acc_20[x] = len(df_40[df_40['error']<0.2])/len(df_40)
                x += 1
            except: 
                ctt.pop('df_40')
                troubleshoot(ctt, df_acc)

        return acc_2, acc_5, acc_10, acc_20, ctt

    acc_2, acc_5, acc_10, acc_20, ctt = troubleshoot(ctt, df_acc)

    acc = pd.DataFrame(
        {'acc_2': acc_2,
         'acc_5': acc_5,
         'acc_10': acc_10,
         'acc_20': acc_20,
        })

    a = []
    if 'df_0' in ctt:
        a.append('<50k')
    if 'df_5' in ctt:
        a.append('50k-100k')
    if 'df_10' in ctt:
        a.append('100k-150k')
    if 'df_15' in ctt:
        a.append('150k-200k')
    if 'df_20' in ctt:
        a.append('200k-250k')
    if 'df_25' in ctt:
        a.append('250k-300k')
    if 'df_30' in ctt:
        a.append('300k-350k')
    if 'df_35' in ctt:
        a.append('350k-400k')
    if 'df_40' in ctt:
        a.append('>400k')

    acc['value bands'] = a

    plt.style.use('ggplot')
    fig, ax = plt.subplots(figsize = (20,10))
    #sns.catplot(x="acc_5", y="percentile", data=acc,ax=ax)
    ax.scatter(acc['acc_2'], acc['value bands'],label='err +-2%', s=80)
    ax.scatter(acc['acc_5'], acc['value bands'],label='err +-5%', s=80)
    ax.scatter(acc['acc_10'], acc['value bands'],label='err +-10%', s=80)
    ax.scatter(acc['acc_20'], acc['value bands'],label='err +-20%', s=80)

    for i in range(0, len(ctt)):
        plt.plot([acc['acc_2'][i], acc['acc_10'][i]], [[i]*5,[i]*5], 'grey')
        plt.plot([acc['acc_5'][i], acc['acc_20'][i]], [[i]*5,[i]*5], 'grey')

    plt.legend(fontsize=15)
    ax.set_xlim(0, 1)
    ax.set_xticklabels([0,20,40,60,80,100], rotation=0, fontsize=15)
    ax.set_yticklabels(acc['value bands'], rotation=0, fontsize=15)
    ax.set_xlabel('Percent of valuations within +-5%, +-20%')
    ax.set_ylabel('Price range of properties £ Pounds')
    plt.title('Accuracy by Value ' + model_name,fontsize=20)
    plt.show()
    imname = 'Accuracy by Price Range.png'
    plt.savefig(imname)
    print('end plot_by_price_range...')
    
    X_misvalued_abs = df_acc[df_acc['error']< 0.2]
    X_misvalued_low = df_acc[df_acc['percent_error'] < -0.2]
    X_misvalued_high = df_acc[df_acc['percent_error'] > 0.2]
    return X_misvalued_low, X_misvalued_high, X_misvalued_abs, df_acc
    

In [17]:
Xx = X
y_true = y['Price_p'].values

In [18]:
y_predicted = pipeline.predict(Xx)

In [19]:
#sanity check the chart below

scores = pipeline.score(X, 
                        y['Price_p'], 
                        objectives=['MAPE',
                                 'MdAPE',
                                 'ExpVariance',
                                 'MaxError',
                                 'MedianAE',
                                 'MSE',
                                 'MAE',
                                 'R2',
                                 'Root Mean Squared Error'])
scores

Out[16]: OrderedDict([('MAPE', 15.426955429844297),
 ('MdAPE', 9.454798377403845),
 ('ExpVariance', 0.892766399577722),
 ('MaxError', 920230.5625),
 ('MedianAE', 20012.52734375),
 ('MSE', 3182616007.2215877),
 ('MAE', 33918.25863464657),
 ('R2', 0.8927646965814696),
 ('Root Mean Squared Error', 56414.67900486174)])

From the plot below we can see that in general:
- Approx 15% of valuations have an absolute error < +-2%
- Approx 35% of valuations have an absolute error < +-5%
- Over 55% of valuations have an absolute error < +-10%
- Approx 75% of valuations have an absolute error < +-20%

In [21]:
#plot and get high error units
X_misvalued_low, X_misvalued_high, X_misvalued_abs, df_acc = _plot_accuracy_by_price_range(y_true, y_predicted, Xx['unit_indx'].values)

end plot_by_price_range...

In [22]:
#find high errors below the true value
X_misvalued_low_units = list(X_misvalued_low['unit_indx'].values)
X_misvalued_low = Xx[Xx['unit_indx'].isin(X_misvalued_low_units)]

#find high errors above the true value
X_misvalued_high_units = list(X_misvalued_high['unit_indx'].values)
X_misvalued_high = Xx[Xx['unit_indx'].isin(X_misvalued_high_units)]

#find high errors above the true value
X_misvalued_abs_units = list(X_misvalued_abs['unit_indx'].values)
X_misvalued_abs = Xx[Xx['unit_indx'].isin(X_misvalued_abs_units)]

columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
X_misvalued_low_t = preprocessor.transform(X_misvalued_low)
X_misvalued_low_t = imputer.transform(X_misvalued_low_t)
X_misvalued_low_t = ohe.transform(X_misvalued_low_t)
X_misvalued_low_t = X_misvalued_low_t[features]

X_misvalued_high_t = preprocessor.transform(X_misvalued_high)
X_misvalued_high_t = imputer.transform(X_misvalued_high_t)
X_misvalued_high_t = ohe.transform(X_misvalued_high_t)
X_misvalued_high_t = X_misvalued_high_t[features]

X_misvalued_abs_t = preprocessor.transform(X_misvalued_abs)
X_misvalued_abs_t = imputer.transform(X_misvalued_abs_t)
X_misvalued_abs_t = ohe.transform(X_misvalued_abs_t)
X_misvalued_abs_t = X_misvalued_abs_t[features]

X_misvalued_abs_t = X_misvalued_abs_t.sample(500)
X_misvalued_high_t = X_misvalued_high_t.sample(500)
X_misvalued_low_t = X_misvalued_low_t.sample(500)

#calc low & high error SHAP values
shap_values_misvalued_low = explainer.shap_values(X_misvalued_low_t.values, check_additivity=False)

In [23]:
shap_values_misvalued_high = explainer.shap_values(X_misvalued_high_t.values, check_additivity=False)

In [24]:
shap_values_misvalued_abs = explainer.shap_values(X_misvalued_abs_t.values, check_additivity=False)

### Figure 5: Understanding High Error Valuations

But what about valuations with over 20% error? Is there any way we can drill down on these valuations to understand what is happening, and perhaps make some corrections to fix, or at the very least, avoid providing valuations when we know the model is most likely to have a high error?

The good news is that we can in fact drill down on these high-error units. To do so, we first calculate the shap_values for only the high-error valuation units. The below SHAP Summary Plot is produced considering *only* high-error valuations. From the SHAP Summary Plot we can see:

- The feature that contributes most to high errors is `Price_p__median` which is an engineered (ie derived) feature built by looking at aggregate statistics about `comparable` units nearby the target property.
- In general, many of the engineered features play a large role in the high error valuations, but it's important to note that they also increase the preformance of the model overall (ie. without them, the model would perform worse). 

However, the overall model Summary Plots(s) and the high-error Summary Plot are very similar, and it's difficult to infer exactly what's happening from the Summary Plots alone.

In [26]:
shap.summary_plot(shap_values_misvalued_abs, X_misvalued_abs_t, features)

### Figure 6: Insights - Absolute Error %

In [28]:
plot_dependence = partial(
    _dependence_plot, 
    feature_names=features,
    category_map={},
)

plot_dependence(
    [ 
     'TOTAL_FLOOR_AREA_e',
     'NUMBER_HEATED_ROOMS_e', 
     'FLOOR_LEVEL_e', 
     'Price_p__median', 
     'Price_p__mean', 
     'TOTAL_FLOOR_AREA_e_minus_mean', 
     'NUMBER_HEATED_ROOMS_e_minus_mean',
     'Latitude_m', 
     'Longitude_m', 
     'FLOOR_LEVEL_e_minus_mean'
    ], 
    shap_values_misvalued_abs, 
    X_misvalued_abs_t,  
    display_features = X_misvalued_abs_t,  
    rotation=33,
    figsize=(47.5, 40), 
    alpha=1, 
    x_jitter=0.5,
    nrows=5,
    ncols=2,
    xlabelfontsize=24,
    xtickfontsize=20,
    xticklabelrotation=0,
    ylabelfontsize=24,
    ytickfontsize=21,
    cbarlabelfontsize=22,
    cbartickfontsize=20,
    cbartickrotation=0
)

### Figure 7: High Error % Overestimation 

From the Summary Plot below we can see that the most important features influencing *overestimation* are very similary to the features influencing absolute error.

In [30]:
shap.summary_plot(shap_values_misvalued_high, X_misvalued_high_t, features)

### Figure 8: Insights - Overestimation Error 

Some interesting patterns with `A` and `B` that can lead to **overestimation error** misvaluation are:
- a
- b

Some interesting patterns with `C` that can lead to **overestimation error** misvaluation are:
- c

Some interesting patterns with `D` that can lead to **overestimation error** misvaluation are:
- d

In [32]:
plot_dependence = partial(
    _dependence_plot, 
    feature_names=features,
    category_map={},
)

plot_dependence(
    [ 
     'TOTAL_FLOOR_AREA_e',
     'NUMBER_HEATED_ROOMS_e', 
     'FLOOR_LEVEL_e', 
     'Price_p__median', 
     'Price_p__mean', 
     'TOTAL_FLOOR_AREA_e_minus_mean', 
     'NUMBER_HEATED_ROOMS_e_minus_mean',
     'Latitude_m', 
     'Longitude_m', 
     'FLOOR_LEVEL_e_minus_mean'
    ], 
    shap_values_misvalued_high, 
    X_misvalued_high_t,  
    display_features = X_misvalued_high_t,  
    rotation=33,
    figsize=(47.5, 40), 
    alpha=1, 
    x_jitter=0.5,
    nrows=5,
    ncols=2,
    xlabelfontsize=24,
    xtickfontsize=20,
    xticklabelrotation=0,
    ylabelfontsize=24,
    ytickfontsize=21,
    cbarlabelfontsize=22,
    cbartickfontsize=20,
    cbartickrotation=0
)

### Figure 9: Underestimation Error

From the Summary Plot below we can see that the most important features influencing *underestimation* are very similary to the features influencing absolute error.

In [34]:
shap.summary_plot(shap_values_misvalued_low, X_misvalued_low_t, features)

### Figure 10: Insights - Underestimation Error 

Some interesting patterns with `A` that can lead to **underestimation error** misvaluation are:
- a

Some interesting patterns with `B` that can lead to **underestimation error** misvaluation are:
- b

Some interesting patterns with `C` that can lead to **underestimation error** misvaluation are:
- c

Some interesting patterns with `D` that can lead to **underestimation error** misvaluation are:
- d

In [36]:
plot_dependence = partial(
    _dependence_plot, 
    feature_names=features,
    category_map={},
)

plot_dependence(
    [ 
     'TOTAL_FLOOR_AREA_e',
     'NUMBER_HEATED_ROOMS_e', 
     'FLOOR_LEVEL_e', 
     'Price_p__median', 
     'Price_p__mean', 
     'TOTAL_FLOOR_AREA_e_minus_mean', 
     'NUMBER_HEATED_ROOMS_e_minus_mean',
     'Latitude_m', 
     'Longitude_m', 
     'FLOOR_LEVEL_e_minus_mean'
    ], 
    shap_values_misvalued_low, 
    X_misvalued_low_t,  
    display_features = X_misvalued_low_t,  
    rotation=33,
    figsize=(47.5, 40), 
    alpha=1, 
    x_jitter=0.5,
    nrows=5,
    ncols=2,
    xlabelfontsize=24,
    xtickfontsize=20,
    xticklabelrotation=0,
    ylabelfontsize=24,
    ytickfontsize=21,
    cbarlabelfontsize=22,
    cbartickfontsize=20,
    cbartickrotation=0
)

### Figure 9: Interesting Feature Interactions

See the [Tree SHAP paper](https://arxiv.org/pdf/1802.03888.pdf) for more details, but briefly, [SHAP interaction values](https://christophm.github.io/interpretable-ml-book/shap.html#shap-interaction-values) are a generalization of SHAP values to higher order interactions. Fast exact computation of pairwise interactions are implemented in the latest version of XGBoost with the pred_interactions flag. With this flag XGBoost returns a matrix for every prediction, where the main effects are on the diagonal and the interaction effects are off-diagonal. The main effects are similar to the SHAP values you would get for a linear model, and the interaction effects captures all the higher-order interactions are divide them up among the pairwise interaction terms. Note that the sum of the entire interaction matrix is the difference between the model's current output and expected output, and so the interaction effects on the off-diagonal are split in half (since there are two of each). When plotting interaction effects the SHAP package automatically multiplies the off-diagonal values by two to get the full interaction effect.

A summary plot of a SHAP interaction value matrix plots a matrix of summary plots with the main effects on the diagonal and the interaction effects off the diagonal.

In [38]:
shap_interaction_values = explainer.shap_interaction_values(Xs)

In [39]:

shap.summary_plot(shap_interaction_values, Xs, 
                  max_display=15
                 )


### Figure 10: Interesting Interactions Cont'd

Now we plot the interaction effects. These effects capture all of the vertical dispersion that was present in the original SHAP plot but is missing from the `main effects plot` (ie the dependence_plot with self-pair (`featureA`,`featureA`)). The plots below show that:.

- Lower `A` interacts with `B` such that ...
- Having a `C` is associated with lower probability of `D` when `E` is greater than approx. `X`

In [41]:
[ 
     'TOTAL_FLOOR_AREA_e',
     'NUMBER_HEATED_ROOMS_e', 
     'FLOOR_LEVEL_e', 
     'Price_p__median', 
     'Price_p__mean', 
     'TOTAL_FLOOR_AREA_e_minus_mean', 
     'NUMBER_HEATED_ROOMS_e_minus_mean',
     'Latitude_m', 
     'Longitude_m', 
     'FLOOR_LEVEL_e_minus_mean'
    ]

plot_dependence = partial(
    _dependence_plot, 
    feature_names=features,
    category_map={},
)

plot_dependence(
    [('TOTAL_FLOOR_AREA_e', 'Price_p__median'),
    ('NUMBER_HEATED_ROOMS_e', 'Price_p__median'), 
    ('FLOOR_LEVEL_e', 'Price_p__median'), 
    ('density_count', 'Price_p__median'), 
    ], 
    shap_interaction_values, 
    Xs, 
    figsize=(30,16.5), 
    rotation=15, 
    ncols=2, 
    nrows=2,
    display_features=Xs,
    xtickfontsize=20,
    xlabelfontsize=20,
    ylabelfontsize=20,
    ytickfontsize=17,
    cbarlabelfontsize=20,
    cbartickfontsize=18,
)

In [42]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
 
# Prepare the dataset
# y_true, y_predicted
test = pd.DataFrame({"Predicted":y_predicted,"Actual":y_true})
test = test.reset_index()
test = test.drop(["index"], axis=1)
 
# plot graphs
fig= plt.figure(figsize=(16,8))
plt.plot(test[:50])
plt.legend(["Actual", "Predicted"])
sns.jointplot(x="Actual", y="Predicted", data=test, kind="reg");